In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.resnet import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet_preprocess_input
from keras.applications.resnet import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
import numpy as np
import pandas as pd
from glob import glob
import matplotlib.pyplot as plt

In [4]:
train_dir = r'chest_xray\train'
test_dir = r'chest_xray\test'
val_dir = r'chest_xray\val'

In [5]:
IMAGE_SIZE = [224, 224]

In [6]:
resnet = ResNet50(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [7]:
for layer in resnet.layers:
    layer.trainable = False

In [8]:
folders = glob('chest_xray/train/*')
print(folders)

['chest_xray/train\\NORMAL', 'chest_xray/train\\PNEUMONIA']


In [9]:
x = Flatten()(resnet.output)

In [10]:
prediction = Dense(len(folders), activation='softmax')(x)

In [11]:
model = Model(inputs=resnet.input, outputs=prediction)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

In [12]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [13]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

In [14]:
test_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

In [15]:
training_set = train_datagen.flow_from_directory(
    'chest_xray/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 5216 images belonging to 2 classes.


In [16]:
test_set = test_datagen.flow_from_directory(
    'chest_xray/test',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 624 images belonging to 2 classes.


In [17]:
val_set = val_datagen.flow_from_directory(
    'chest_xray/val',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 16 images belonging to 2 classes.


In [18]:
early_stopping = EarlyStopping(
    min_delta=0.001,  # minimum amount of change to count as an improvement
    patience=20,  # how many epochs to wait before stopping
    restore_best_weights=True,
)

In [19]:
r = model.fit(
    training_set,
    validation_data=val_set,
    epochs=500,
    steps_per_epoch=len(training_set),
    validation_steps=len(val_set),
    callbacks=[early_stopping]
)

Epoch 1/500
163/163 [==============================] - 254s 2s/step - loss: 0.8460 - accuracy: 0.7941 - val_loss: 0.5221 - val_accuracy: 0.6875
Epoch 2/500
163/163 [==============================] - 263s 2s/step - loss: 0.3471 - accuracy: 0.8612 - val_loss: 0.4740 - val_accuracy: 0.8125
Epoch 3/500
163/163 [==============================] - 261s 2s/step - loss: 0.3364 - accuracy: 0.8641 - val_loss: 0.5203 - val_accuracy: 0.7500
Epoch 4/500
163/163 [==============================] - 219s 1s/step - loss: 0.3270 - accuracy: 0.8735 - val_loss: 1.4201 - val_accuracy: 0.6250
Epoch 5/500
163/163 [==============================] - 224s 1s/step - loss: 0.2838 - accuracy: 0.8900 - val_loss: 2.5210 - val_accuracy: 0.5625
Epoch 6/500
163/163 [==============================] - 224s 1s/step - loss: 0.2613 - accuracy: 0.8961 - val_loss: 0.4729 - val_accuracy: 0.8125
Epoch 7/500
163/163 [==============================] - 224s 1s/step - loss: 0.2862 - accuracy: 0.8930 - val_loss: 0.4306 - val_accuracy:

In [20]:
metrics_data = []

In [21]:
metrics = {
        'Model': 'ResNet-18',
        'Training Loss': r.history['loss'][-1],
        'Validation Loss': r.history['val_loss'][-1],
        'Training Accuracy': r.history['accuracy'][-1],
        'Validation Accuracy': r.history['val_accuracy'][-1]
    }
    

In [22]:
metrics_data.append(metrics)

In [23]:
metrics_df = pd.DataFrame(metrics_data)

In [25]:
metrics_df.to_csv('Metrics/Res-Net18model_metrics.csv', index=False)

In [ ]:
model.save_weights(r'training_1/mode_resnet18_weights.h5')
model.save(r'training_1/model_resnet18.h5')